# theanoでも線形回帰をやってみる

詳しくはtensorflowのnotebookを参照

In [1]:
import numpy as np
import theano
import theano.tensor as T

TheanoでもTheanoのVariableを定義する。Theanoではscalorやvectorやmatrixを明確に定義しないといけない。
また、更新するような変数についてはsharedで定義する。これに関しては、scalorなどの定義は不要である。
また、微分は、`T.grad`で計算でき、第一引数には対象となる関数、第二引数には微分したい変数を入れる。

In [2]:
x = T.vector("x")
y = T.vector("y")
w = theano.shared(np.random.rand(), name="w")
print(w.get_value())
b = theano.shared(0., name="b")
print(b.get_value())
prediction = w * x  + b
cost = T.mean(T.sqr(y - prediction))
gw,gb =  T.grad(cost, [w, b])

0.8245496255283107
0.0


Theanoでは関数をコンパイルする必要がある。`theano.function`で、第一引数は入力変数をリストで、第二引数は出力関数を入れる。

In [3]:
predict = theano.function([x], prediction)

これで、predictという関数が作られ、引数を与えることで、実行できる

In [4]:
predict([1,2])

array([ 0.82454963,  1.64909925])

さて、theanoでは、最急降下法などの実装がないので、自分でパラメーターを更新する必要がある。
上記で求めておいた、各変数での偏微分とlearning_rateで、各変数を動かすことで、costが最小となるようにする。

In [5]:
learning_rate = .5
updates = ((w, w - learning_rate*gw), (b, b - learning_rate*gb))
train = theano.function([x, y], prediction, updates = updates)
train([1,2],[1,2])

array([ 0.82454963,  1.64909925])

In [6]:
x_data = np.random.rand(100).astype("float32")
y_data = x_data * 0.1 + 0.3
print("x_data -> ", x_data[:5])
print("y_data -> ", y_data[:5])

x_data ->  [ 0.18078901  0.58026898  0.88836986  0.57172632  0.12188106]
y_data ->  [ 0.31807891  0.35802692  0.38883701  0.35717264  0.31218812]


In [7]:
for step in range(201):
    train(x_data, y_data)
    if step % 20 == 0:
        print(step, w.get_value(), b.get_value())

0 0.8353584227469867 -0.3327954443041955
20 0.3032191396472578 0.1814280739345419
40 0.15011642487238233 0.2707586641215669
60 0.1123593516704801 0.29278872176614046
80 0.10304797806890166 0.2982216136668453
100 0.10075167512495938 0.29956143378356725
120 0.10018537766596107 0.29989185043421696
140 0.10004572152306668 0.2999733353751161
160 0.10001128054520564 0.29999343059604483
180 0.10000278696281509 0.2999983863326474
200 0.10000069233756613 0.29999960848020985


上手く学習できている